In [ ]:
import tensorflow as tf
import tensorflow.python.keras
#import keras
from tensorflow.python.keras import backend as K
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
print(tf.__version__)
#print(keras.__version__)
import cv2  # for image processing
from sklearn.model_selection import train_test_split
import scipy.io
import os
import h5py
from arts_preprocess_utils import load_dataset, get_image
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import scipy
#from scipy.spatial.distance import directed_hausdorff, cosine
from IPython import display

In [ ]:
# !!! remember to clear session/graph if you rebuild your graph to avoid out-of-memory errors !!!
def reset_tf_session():
    K.clear_session()
    s = K.get_session()
    return s

In [ ]:
reset_tf_session()

## Load data

**Metada**

In [ ]:
df_artworks = pd.read_csv('/root/work/datasets/train_mayors_style.csv')
df_artworks.head()

**Image shape**

In [ ]:
#According to the dataset input used during autoencoder training

img_Height = 128
img_Width = 128
chanels= 3
N_CLASSES = 1

### Build encoder

In [ ]:
from models import Generator_model_complex, Discriminator_model

def build_deep_autoencoder(img_shape, code_size, filters = 40):
    
    encoder_model = Discriminator_model(filters=filters, code_shape=code_size, include_top=False)
    encoder = encoder_model.get_model(img_shape, N_CLASSES, False)
    
    decoder_model = Generator_model_complex(filters=filters, code_shape= (1,1,code_size))
    decoder = decoder_model.get_model(img_shape)
    
    return encoder, decoder

In [ ]:
#Denoisy create a code with 300 features
encoder, decoder = build_deep_autoencoder((img_Height, img_Width, 3), 300)
#Wasserstein create a code with 100 features
#encoder, decoder = build_deep_autoencoder((img_Height, img_Width, 3), 100)

In [ ]:
#Denoisy encoder
encoder.load_weights('./weigths/denoisy_encoder01.h5')

In [ ]:
#Wasserstein encoder
encoder.load_weights('./weigths/wasserstein_encoder01.h5')
decoder.load_weights('./weigths/wasserstein_decoder01.h5')

In [ ]:
encoder.summary()

**Save encoder model to use for flask application**

In [ ]:
encoder.save('wasserstein_encoder.h5')

**Save decoder model to use for flask application**

In [ ]:
decoder.save('wasserstein_decoder.h5')

### Extract feature from output layer in VGG19

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np

base_model = VGG19(input_shape= (img_Height, img_Width, 3) ,weights='imagenet', include_top=False)
encoder = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

In [ ]:
encoder.layers[-1].output.shape

## Encode artworks

In [ ]:
import zipfile

#dataset from Kaggle
filezip = zipfile.ZipFile('/root/work/datasets/train.zip', 'r')

In [ ]:
import os

def encode_artworks(x, encoder):

    try:
        img_path = filezip.extract('train/' + x)
        # Images
        image = get_image(img_path, img_Height=img_Height, img_Width=img_Width)

        # Encode
        code = encoder.predict(image)
        
        try:
            os.remove(img_path)
        except OSError as e:  ## if failed, report it back to the user ##
            print ("Error: %s - %s." % (e.filename, e.strerror))
            
        return code.reshape((-1,))

    except:
        print("Problem with file " + x)
        return None


In [ ]:
df_artworks['code'] = df_artworks['filename'].apply(encode_artworks, encoder=encoder)

In [ ]:
df_artworks.head()

In [ ]:
df_artworks['code'].isnull().value_counts()

In [ ]:
#Drop null values
df_artworks = df_artworks.dropna(subset=['code'])
df_artworks.shape

## Save values

In [ ]:
df_artworks.to_csv('/root/work/datasets/train_mayors_style_w_encoded.csv', index=False)

In [ ]:
encodes_list = list(df_artworks['code'])

In [ ]:
encode_matrix = np.stack(encodes_list)
encode_matrix.shape

In [ ]:
np.save('/root/work/datasets/train_mayors_style_encoded_vgg19', encode_matrix)

In [ ]:
df_toy = df_artworks[0:10]
df_toy

In [ ]:
df_toy['code'] = df_toy['filename'].apply(encode_artworks, encoder=encoder)

In [ ]:
df_toy